## Data colleciton and selection
In this notebook, the raw data of interest from the last manikin tests are gathered and filtered. Only the selected steady state 15' intervals are kept and stored into CSV files. There will be 7 (= number configurations) CSV files per HOBO sensor (including relative humidity) and cube sensor, one for the flowmeter, siemens log, manikin.

In [1]:
import pandas as pd
from datetime import datetime
import glob
import os
import re

#### HOBO sensors

In [84]:
# Functions
hoboAcronyms = ['AH1', 'AH2', 'AH3',
                'BH1', 'BH2', 'BH3',
                'CH1', 'CH2', 'CH3',
                'DH1', 'DH2', 'DH3']

def create_hobo_DFs(date, test):
    ''' test is a string looking like 8.1 or 9.2
        date is the date in the folder nomenclature, eg 20231106
        returns a pandas dataframes list for all hobo sensors of one test
    '''

    file_path = 'Y:/PROJECTS/ManikinStudies/2_2023_Diego Houtart/040. Data/010. Raw/{} - Manikin Test {}/HOBO/{}_{}.csv'
    hoboCSVFiles = [file_path.format(date, test, date, hoboAcronym) for hoboAcronym in hoboAcronyms]


    # Specify the date and time format
    date_time_format = '%m.%d.%y %I:%M:%S %p' 

    dtypes = {
        0: 'int',
        1: 'float64',
        2: 'float64',
        3: 'float64',
        4: 'float64',
        5: 'float64',
        6: 'str',
        7: 'str',
        8: 'str'
    }

    # dfs will be a list of pandas dataframes, each element corresponding to a hobo sensor
    # Skip rows that are not multiples of 10 and not the header
    # and avoid taking the first row which is useless in the file
    dfs = [pd.read_csv(file,
                    dtype=dtypes,
                    skiprows=1, 
                    parse_dates=['Date Heure, GMT+01:00'],
                    date_format=date_time_format
                    )
        for file in hoboCSVFiles]


    for df in dfs:
        df.drop(df.tail(1).index,inplace=True) #drop last rows that can be not well written
        cols_to_drop = df.filter(like='Hôte connecté').columns.tolist() + \
                        df.filter(like='Arrêté').columns.tolist() + \
                        df.filter(like='Fin de fichier').columns.tolist() # drop recording-related columns
        df.drop(columns=cols_to_drop, inplace=True)
        df.drop(columns=df.columns[-1:], axis=1,  inplace=True) #drop last useless column anemometer temperature
        df.drop(columns=df.columns[0], inplace=True) #drop first column with index, not useful
        df.columns = ['Date and Time', 'T_a [°C]','T_g [°C]', 'v_a [mA]'] #rename columns
        df['Date and Time'] = pd.to_datetime(df['Date and Time'], format=date_time_format)
    
    return dfs

def save_hobo_period_csv(hobo_dfs, configName, start_time, end_time):
    ''' Save a list of csv files corresponding to one configuration 
        during its specified time interval between start_time and end_time.
        Each csv of the list correspond to a hobo sensor.
    '''
    
    filtered_dfs = [] 
    for i in range(len(hobo_dfs)):
        mask = (hobo_dfs[i]['Date and Time'] >= start_time) & (hobo_dfs[i]['Date and Time'] <= end_time)
        filtered_dfs.append(hobo_dfs[i][mask])
        

    for filtered_df,hoboAcronym in zip(filtered_dfs, hoboAcronyms):
        filtered_df.to_csv(f'HOBO/{configName}_{hoboAcronym}.csv', index=False, sep='\t')




In [85]:
# Dataframes
hobo_sensors_8_1 = create_hobo_DFs('20231106', '8.1')
hobo_sensors_9_1 = create_hobo_DFs('20231108', '9.1')

# Save CSV
save_hobo_period_csv(hobo_sensors_9_1, 'floorConfig0', '2023-11-06 22:45:00', '2023-11-06 23:00:00')
save_hobo_period_csv(hobo_sensors_9_1, 'floorConfig1', '2023-11-07 00:45:00', '2023-11-07 01:00:00')
save_hobo_period_csv(hobo_sensors_9_1, 'floorConfig2a', '2023-11-07 16:45:00', '2023-11-07 17:00:00')
save_hobo_period_csv(hobo_sensors_8_1, 'floorConfig2b', '2023-11-05 05:25:00', '2023-11-05 05:40:00')
save_hobo_period_csv(hobo_sensors_8_1, 'floorConfig3', '2023-11-05 12:25:00', '2023-11-05 12:40:00')
save_hobo_period_csv(hobo_sensors_9_1, 'floorConfig4', '2023-11-08 00:45:00', '2023-11-08 01:00:00')
save_hobo_period_csv(hobo_sensors_9_1, 'floorConfig6', '2023-11-08 05:45:00', '2023-11-08 06:00:00')


#### HOBO relative humidity sensor

In [86]:
# Functions
def create_hobo_RH_DF(date, test):
    ''' test is a string looking like 8.1 or 9.2
        date is the date in the folder nomenclature, eg 20231106
        returns a pandas dataframes list for all hobo sensors of one test
    '''

    file_path = 'Y:/PROJECTS/ManikinStudies/2_2023_Diego Houtart/040. Data/010. Raw/{} - Manikin Test {}/HOBO_RH/{}_RH1.csv'
    hobo_RH_CSV_File = file_path.format(date, test, date)


    # Specify the date and time format
    date_time_format = '%m.%d.%y %I:%M:%S %p' 

    dtypes = {
        0: 'int',
        1: 'float64',
        2: 'float64',
        3: 'float64',
        6: 'str',
        7: 'str',
        8: 'str'
    }

    # dfs will be a list of pandas dataframes, each element corresponding to a hobo sensor
    # Skip rows that are not multiples of 10 and not the header
    # and avoid taking the first row which is useless in the file
    df = pd.read_csv(hobo_RH_CSV_File,
                    dtype=dtypes,
                    skiprows=1, 
                    parse_dates=['Date Heure, GMT+01:00'],
                    date_format=date_time_format
                    )


    df.drop(df.tail(1).index,inplace=True) #drop last rows that can be not well written
    cols_to_drop = df.filter(like='Hôte connecté').columns.tolist() + \
                    df.filter(like='Arrêté').columns.tolist() + \
                    df.filter(like='Fin de fichier').columns.tolist() # drop recording-related columns
    df.drop(columns=cols_to_drop, inplace=True)
    df.drop(columns=df.columns[0], inplace=True) #drop first column with index, not useful
    df.columns = ['Date and Time', 'T_a [°C]','RH [°%]'] #rename columns
    df['Date and Time'] = pd.to_datetime(df['Date and Time'], format=date_time_format)
    
    return df

def save_hobo_rh_period_csv(hobo_rh_df, configName, start_time, end_time):
    ''' Save a list of csv files corresponding to one configuration 
        during its specified time interval between start_time and end_time.
        Each csv of the list correspond to a hobo sensor.
    '''
    

    mask = (hobo_rh_df['Date and Time'] >= start_time) & (hobo_rh_df['Date and Time'] <= end_time)
    filtered_df = hobo_rh_df[mask]
        
    filtered_df.to_csv(f'HOBO_RH/{configName}_RH.csv', index=False, sep='\t')

In [87]:
# Dataframe
hobo_rh_8_1 = create_hobo_RH_DF('20231106', '8.1')

# Save CSV
save_hobo_rh_period_csv(hobo_rh_8_1, 'floorConfig2b', '2023-11-05 05:25:00', '2023-11-05 05:40:00')
save_hobo_rh_period_csv(hobo_rh_8_1, 'floorConfig3', '2023-11-05 12:25:00', '2023-11-05 12:40:00')


#### HOBO Airflow

In [88]:
# Functions
hobo_AF_Acronyms = ['Extract', 'Supply']

def create_hobo_AF_DFs(date, test):
    ''' test is a string looking like 8.1 or 9.2
        date is the date in the folder nomenclature, eg 20231106
        returns a pandas dataframes list for all hobo sensors of one test
    '''

    file_path = 'Y:/PROJECTS/ManikinStudies/2_2023_Diego Houtart/040. Data/010. Raw/{} - Manikin Test {}/HOBO_AIRFLOW/{}_{}.csv'
    hobo_AF_CSVFiles = [file_path.format(date, test, date, hobo_AF_Acronym) for hobo_AF_Acronym in hobo_AF_Acronyms]


    # Specify the date and time format
    date_time_format = '%m.%d.%y %I:%M:%S %p' 

    dtypes = {
        0: 'int',
        1: 'float64',
        2: 'float64',
        6: 'str',
        7: 'str',
        8: 'str'
    }

    # dfs will be a list of pandas dataframes, each element corresponding to a hobo sensor
    # Skip rows that are not multiples of 10 and not the header
    # and avoid taking the first row which is useless in the file
    dfs = [pd.read_csv(file,
                    dtype=dtypes,
                    skiprows=1, 
                    parse_dates=['Date Heure, GMT+01:00'],
                    date_format=date_time_format
                    )
        for file in hobo_AF_CSVFiles]


    for df in dfs:
        df.drop(df.tail(1).index,inplace=True) #drop last rows that can be not well written
        cols_to_drop = df.filter(like='Hôte connecté').columns.tolist() + \
                        df.filter(like='Arrêté').columns.tolist() + \
                        df.filter(like='Fin de fichier').columns.tolist() # drop recording-related columns
        df.drop(columns=cols_to_drop, inplace=True)
        df.drop(columns=df.columns[0], inplace=True) #drop first column with index, not useful
        df.columns = ['Date and Time', 'T_a [°C]'] #rename columns
        df['Date and Time'] = pd.to_datetime(df['Date and Time'], format=date_time_format)
    
    return dfs

def save_hobo_AF_period_csv(hobo_AF_dfs, configName, start_time, end_time):
    ''' Save a list of csv files corresponding to one configuration 
        during its specified time interval between start_time and end_time.
        Each csv of the list correspond to a hobo sensor.
    '''
    
    filtered_dfs = [] 
    for i in range(len(hobo_AF_dfs)):
        mask = (hobo_AF_dfs[i]['Date and Time'] >= start_time) & (hobo_AF_dfs[i]['Date and Time'] <= end_time)
        filtered_dfs.append(hobo_AF_dfs[i][mask])
        

    for filtered_df,hobo_AF_Acronym in zip(filtered_dfs, hobo_AF_Acronyms):
        filtered_df.to_csv(f'HOBO_AF/{configName}_{hobo_AF_Acronym}.csv', index=False, sep='\t')




In [89]:
# Dataframes
hobo_AF_sensors_8_1 = create_hobo_AF_DFs('20231106', '8.1')
hobo_AF_sensors_9_1 = create_hobo_AF_DFs('20231108', '9.1')

# Save CSV
save_hobo_AF_period_csv(hobo_AF_sensors_9_1, 'floorConfig0', '2023-11-06 22:45:00', '2023-11-06 23:00:00')
save_hobo_AF_period_csv(hobo_AF_sensors_9_1, 'floorConfig1', '2023-11-07 00:45:00', '2023-11-07 01:00:00')
save_hobo_AF_period_csv(hobo_AF_sensors_9_1, 'floorConfig2a', '2023-11-07 16:45:00', '2023-11-07 17:00:00')
save_hobo_AF_period_csv(hobo_AF_sensors_8_1, 'floorConfig2b', '2023-11-05 05:25:00', '2023-11-05 05:40:00')
save_hobo_AF_period_csv(hobo_AF_sensors_8_1, 'floorConfig3', '2023-11-05 12:25:00', '2023-11-05 12:40:00')
save_hobo_AF_period_csv(hobo_AF_sensors_9_1, 'floorConfig4', '2023-11-08 00:45:00', '2023-11-08 01:00:00')
save_hobo_AF_period_csv(hobo_AF_sensors_9_1, 'floorConfig6', '2023-11-08 05:45:00', '2023-11-08 06:00:00')

#### CUBES sensors

In [90]:
# Functions

cubeAcronyms = ['BC1', 'BC2', 'CC1', 'DC1', 'DC2']

def create_cubes_DFs(date, test):
    ''' test is a string looking like 8.1 or 9.2
        date is the date in the folder nomenclature, eg 20231106
        returns a pandas dataframes list for all hobo sensors of one test
    '''

    file_path = 'Y:/PROJECTS/ManikinStudies/2_2023_Diego Houtart/040. Data/010. Raw/{} - Manikin Test {}/CUBES/{}_{}.csv'
    cubeCSVFiles = [file_path.format(date, test, date, cubeAcronym) for cubeAcronym in cubeAcronyms]


    # Specify the date and time format
    date_time_format = "%d-%m-%Y  %H:%M:%S"

    pat = r".+?{}?$".format("([^,]+),"*19)
   
    dfs = [pd.read_csv(file,
                    header=0,
                    skiprows=range(1, 3),
                    on_bad_lines = 'skip',
                    sep=pat,
                    engine='python'
                    ).dropna(how="all", axis=1)
        for file in cubeCSVFiles]

    dfs_copies = []
    for df, i in zip(dfs, range(len(dfs))):
        columns_to_drop = df.filter(like='Unnamed').columns
        df.drop(columns=columns_to_drop, inplace=True)
        df.rename(columns=lambda x: x.strip(), inplace=True)
        df['DD-MM-YYYY'] = df['DD-MM-YYYY'].str.strip(' ')
        df['HH:MM:SS'] = df['HH:MM:SS'].str.strip(' ')
        df = df[~df.apply(lambda row: row.astype(str).str.contains('DD-MM-YYYY')).any(axis=1)]
        df.insert(0, 'Date and Time', pd.to_datetime(df['DD-MM-YYYY'] + ' ' + df['HH:MM:SS'], format=date_time_format, errors='coerce'))
        df.dropna(axis=0,inplace=True)
        df.drop(['DD-MM-YYYY', 'HH:MM:SS'], axis=1, inplace=True)
        df.drop(df.tail(1).index,inplace=True) #drop last rows that can be not well written
        dfs_copies.append(df.copy())
    return dfs_copies



def save_cube_period_csv(cube_dfs, configName, start_time, end_time):
    ''' Save a list of csv files corresponding to one configuration 
        during its specified time interval between start_time and end_time.
        Each csv of the list correspond to a cube sensor.
    '''
    
    filtered_dfs = [] 
    for i in range(len(cube_dfs)):
        mask = (cube_dfs[i]['Date and Time'] >= start_time) & (cube_dfs[i]['Date and Time'] <= end_time)
        filtered_dfs.append(cube_dfs[i][mask])
        

    for filtered_df,cubeAcronym in zip(filtered_dfs, cubeAcronyms):
        filtered_df.to_csv(f'CUBES/{configName}_{cubeAcronym}.csv', index=False, sep='\t')




In [91]:
# Dataframe
cube_sensors_8_1 = create_cubes_DFs('20231106', '8.1')
cube_sensors_9_1 = create_cubes_DFs('20231108', '9.1')

# CSV
save_cube_period_csv(cube_sensors_9_1, 'floorConfig0', '2023-11-06 22:45:00', '2023-11-06 23:00:00')
save_cube_period_csv(cube_sensors_9_1, 'floorConfig1', '2023-11-07 00:45:00', '2023-11-07 01:00:00')
save_cube_period_csv(cube_sensors_9_1, 'floorConfig2a', '2023-11-07 16:45:00', '2023-11-07 17:00:00')
save_cube_period_csv(cube_sensors_8_1, 'floorConfig2b', '2023-11-05 05:25:00', '2023-11-05 05:40:00')
save_cube_period_csv(cube_sensors_8_1, 'floorConfig3', '2023-11-05 12:25:00', '2023-11-05 12:40:00')
save_cube_period_csv(cube_sensors_9_1, 'floorConfig4', '2023-11-08 00:45:00', '2023-11-08 01:00:00')
save_cube_period_csv(cube_sensors_9_1, 'floorConfig6', '2023-11-08 05:45:00', '2023-11-08 06:00:00')



C:\Users\houtart\AppData\Local\Temp\ipykernel_20976\2770424432.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0,inplace=True)
C:\Users\houtart\AppData\Local\Temp\ipykernel_20976\2770424432.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['DD-MM-YYYY', 'HH:MM:SS'], axis=1, inplace=True)
C:\Users\houtart\AppData\Local\Temp\ipykernel_20976\2770424432.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df

#### FLOWMETER

In [92]:
# Functions

flowmeter_acronyms = ['A', 'B']
def create_flowmeter_DFs(date, test):
    ''' Create and returns two dataframe for the specified date and test. 
        One df for A and one for B measurements combined
        Flowmeter were in summer time, 1 hour is retrieved to timestamps
    '''
    a_header = ['Date and Time', 'Chanel', 'Flow (L/s)', 'Q_POS', 'Q_NEG', 'T1', 'T2', 'SSPEED', 'AMP', 'SCNR']
    b_header = ['Date and Time', 'Chanel', 'Flow (L/s)', 'Q_POS', 'Q_NEG', 'T3', 'T4', 'SSPEED', 'AMP', 'SCNR']

    date_time_format = "%Y/%m/%d %H:%M:%S,%f"

    file_path = fr'Y:\PROJECTS\ManikinStudies\2_2023_Diego Houtart\040. Data\010. Raw\{date} - Manikin Test {test}\FLOWMETER'
    
    all_files = glob.glob(os.path.join(file_path, "*.txt")) # we make a list of all log files of the flowmeter
    dfs_a = []
    dfs_b = []
    for f in all_files: # we create one dataframe for A and one for B, combining all files
        with open(f, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            # Data is encoded in a unusual way in the txt
            data = [list(map(lambda x: x.strip(' :'), re.split(r'[;\t]', line.strip()))) for line in lines]
            data_a = [data[i] for i in range(len(data)) if i % 2 == 0]
            data_b = [data[i] for i in range(len(data)) if i % 2 != 0]
            df_a = pd.DataFrame(data_a, columns=a_header)
            df_b = pd.DataFrame(data_b, columns=b_header)
            dfs_a.append(df_a); dfs_b.append(df_b)
    combined_df_a = pd.concat(dfs_a, ignore_index=True)
    combined_df_b = pd.concat(dfs_b, ignore_index=True)


    cols_to_drop = ['Q_POS', 'Q_NEG', 'SSPEED', 'AMP', 'SCNR']
    for df in [combined_df_a, combined_df_b]:
        df.drop(columns=cols_to_drop, inplace=True)
        df['Date and Time'] = pd.to_datetime(df['Date and Time'], format=date_time_format, errors='coerce')
        df['Date and Time'] = df['Date and Time'] - pd.Timedelta(hours=1)
        df['Date and Time'] = df['Date and Time'].astype('datetime64[s]')

    return [combined_df_a, combined_df_b]

def save_flowmeter_period_csv(flowmeter_dfs, configName, start_time, end_time):
    ''' Save a list of csv files corresponding to one configuration 
        during its specified time interval between start_time and end_time.
        Each csv of the list correspond to a cube sensor.
    '''
    
    filtered_dfs = [] 
    for i in range(len(flowmeter_dfs)):
        mask = (flowmeter_dfs[i]['Date and Time'] >= start_time) & (flowmeter_dfs[i]['Date and Time'] <= end_time)
        filtered_dfs.append(flowmeter_dfs[i][mask])
        

    for filtered_df,flowmeter_acronym in zip(filtered_dfs, flowmeter_acronyms):
        filtered_df.to_csv(f'FLOWMETER/{configName}_{flowmeter_acronym}.csv', index=False, sep='\t')





In [93]:
# Dataframes
flowmeters_8_1 = create_flowmeter_DFs('20231106', '8.1')
flowmeters_9_1 = create_flowmeter_DFs('20231108', '9.1')

# CSV
save_flowmeter_period_csv(flowmeters_9_1, 'floorConfig0', '2023-11-06 22:45:00', '2023-11-06 23:00:00')
save_flowmeter_period_csv(flowmeters_9_1, 'floorConfig1', '2023-11-07 00:45:00', '2023-11-07 01:00:00')
save_flowmeter_period_csv(flowmeters_9_1, 'floorConfig2a', '2023-11-07 16:45:00', '2023-11-07 17:00:00')
save_flowmeter_period_csv(flowmeters_8_1, 'floorConfig2b', '2023-11-05 05:25:00', '2023-11-05 05:40:00')
save_flowmeter_period_csv(flowmeters_8_1, 'floorConfig3', '2023-11-05 12:25:00', '2023-11-05 12:40:00')
save_flowmeter_period_csv(flowmeters_9_1, 'floorConfig4', '2023-11-08 00:45:00', '2023-11-08 01:00:00')
save_flowmeter_period_csv(flowmeters_9_1, 'floorConfig6', '2023-11-08 05:45:00', '2023-11-08 06:00:00')


#### MANIKIN

In [11]:
# Functions
def create_manikin_DF(date, test):
    ''' test is a string looking like 8.1 or 9.2
        date is the date in the folder nomenclature, eg 20231106
        returns a pandas dataframes list for all hobo sensors of one test
    '''

    file_path = f'Y:/PROJECTS/ManikinStudies/2_2023_Diego Houtart/040. Data/010. Raw/{date} - Manikin Test {test}/MANIKIN/{date}_Manikin.csv'


    # Specify the date and time format
    date_time_format = '%m/%d/%Y %I:%M:%S %p' 


    manikin_df = pd.read_csv(file_path,\
        encoding='ansi',\
        sep='\t',\
        header=[0,1,2],
        skiprows=3)

    header_tuples = [
        ('Date and Time', ' ', ' '),
        ('Runtime', ' ', 'sec')]
    next_header_tuples = [(a, b, c) \
        for a in ['All','Group A','Group B'] \
        for b, c in zip(
            ['T','P','Teq','PMV','PPD','SET*','ET*'],
            ['degC','W/m²','degC', ' ','%','degC','degC'])]

    header_tuples.extend(next_header_tuples)
    next_header_tuples = [(a, b, c) \
        for a in [
            'L. Foot','R. Foot','L. Foreleg','R. Foreleg',\
            'L. Front Thigh','R. Front Thigh','L. Back Thigh',\
            'R. Back Thigh','Pelvis','Back Side','Head','Schll',\
            'L. Hand','R. Hand','L. Forearm','R. Forearm',\
            'L. Upper Arm','R. Upper Arm','Chest',\
            'Low Back','Upper Back','Stomach'] \
        for b, c in zip(
            ['T','P','Teq','MTV'],
            ['degC','W/m²','degC', ' '])]

    header_tuples.extend(next_header_tuples)
    next_tuples = [
        ('Stability', ' ', ' '),
        ('','','')]
    header_tuples.extend(next_tuples)
    manikin_df.columns = pd.MultiIndex.from_tuples(header_tuples)
    manikin_df = manikin_df.iloc[:,:-1] # Delete last column (all N/A)

    manikin_df = manikin_df.drop(['Runtime', 'Group A', 'Group B'], axis=1, level=0)
    manikin_df['Date and Time', ' ', ' '] = pd.to_datetime(manikin_df['Date and Time', ' ', ' '], format=date_time_format, errors='raise')
    return manikin_df


def save_manikin_period_csv(manikin_df, configName, start_time, end_time):
    ''' Save a csv files corresponding to one configuration 
        during its specified time interval between start_time and end_time.
    '''
    

    mask = (manikin_df['Date and Time', ' ', ' '] >= start_time) & (manikin_df['Date and Time', ' ', ' '] <= end_time)
    filtered_df = manikin_df[mask]
        
    filtered_df.to_csv(f'MANIKIN/{configName}_manikin.csv', index=False, sep='\t')

In [12]:
# Dataframes
manikin_8_1 = create_manikin_DF('20231106', '8.1')
manikin_9_1 = create_manikin_DF('20231108', '9.1')

# CSV
save_manikin_period_csv(manikin_9_1, 'floorConfig0', '2023-11-06 22:45:00', '2023-11-06 23:00:00')
save_manikin_period_csv(manikin_9_1, 'floorConfig1', '2023-11-07 00:45:00', '2023-11-07 01:00:00')
save_manikin_period_csv(manikin_9_1, 'floorConfig2a', '2023-11-07 16:45:00', '2023-11-07 17:00:00')
save_manikin_period_csv(manikin_8_1, 'floorConfig2b', '2023-11-05 05:25:00', '2023-11-05 05:40:00')
save_manikin_period_csv(manikin_8_1, 'floorConfig3', '2023-11-05 12:25:00', '2023-11-05 12:40:00')
save_manikin_period_csv(manikin_9_1, 'floorConfig4', '2023-11-08 00:45:00', '2023-11-08 01:00:00')
save_manikin_period_csv(manikin_9_1, 'floorConfig6', '2023-11-08 05:45:00', '2023-11-08 06:00:00')

#### SIEMENS

In [16]:
# Functions
def create_siemens_DF(date, test):
    ''' test is a string looking like 8.1 or 9.2
        date is the date in the folder nomenclature, eg 20231106
        returns a pandas dataframes list for all hobo sensors of one test
    '''

    date_time_format = '%Y-%m-%d %H:%M:%S'
    file_path = 'Y:/PROJECTS/ManikinStudies/2_2023_Diego Houtart/040. Data/010. Raw/{} - Manikin Test {}/SIEMENS/{}_siemens_log.csv'
    siemens_CSV_File = file_path.format(date, test, date)

    # dfs will be a list of pandas dataframes, each element corresponding to a hobo sensor
    # Skip rows that are not multiples of 10 and not the header
    # and avoid taking the first row which is useless in the file
    df = pd.read_csv(siemens_CSV_File,
                    header=[0]
                    )


    df.drop(df.tail(1).index,inplace=True) #drop last rows that can be not well written
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], format=date_time_format)
    df.rename(columns={'Timestamp': 'Date and Time'}, inplace=True)    
    return df

def save_siemens_period_csv(siemens_df, configName, start_time, end_time):
    ''' Save a list of csv files corresponding to one configuration 
        during its specified time interval between start_time and end_time.
        Each csv of the list correspond to a hobo sensor.
    '''
    

    mask = (siemens_df['Date and Time'] >= start_time) & (siemens_df['Date and Time'] <= end_time)
    filtered_df = siemens_df[mask]
        
    filtered_df.to_csv(f'SIEMENS/{configName}_siemens.csv', index=False, sep='\t')

In [17]:
siemens_8_1 = create_siemens_DF('20231106', '8.1')
siemens_9_1 = create_siemens_DF('20231108', '9.1')



save_siemens_period_csv(siemens_9_1, 'floorConfig0', '2023-11-06 22:45:00', '2023-11-06 23:00:00')
save_siemens_period_csv(siemens_8_1, 'floorConfig2b', '2023-11-05 05:25:00', '2023-11-05 05:40:00')
save_siemens_period_csv(siemens_8_1, 'floorConfig3', '2023-11-05 12:25:00', '2023-11-05 12:40:00')

#### THERMOCOUPLES

In [39]:
# Functions
def create_TC_DF(date, test):
    ''' test is a string looking like 8.1 or 9.2
        date is the date in the folder nomenclature, eg 20231106
        returns a pandas dataframe for the thermocouples of one test
    '''

    date_time_format = '%d.%m.%Y %H.%M.%S'
    file_path = 'Y:/PROJECTS/ManikinStudies/2_2023_Diego Houtart/040. Data/010. Raw/{} - Manikin Test {}/THERMOCOUPLES/{}_TC.txt'
    siemens_CSV_File = file_path.format(date, test, date)

    # dfs will be a list of pandas dataframes, each element corresponding to a hobo sensor
    # Skip rows that are not multiples of 10 and not the header
    # and avoid taking the first row which is useless in the file
    TC_cols_file_path = r'Y:\PROJECTS\ManikinStudies\2_2023_Diego Houtart\040. Data\000. External\Thermocouples nomenclature\thermocouples_order.txt'
    with open(TC_cols_file_path) as TC_cols_file:
            TC_cols = TC_cols_file.read().split(', ')
    TC_cols = ['DD.MM.YYYY', 'HH.MM.SS'] + TC_cols
    df = pd.read_csv(siemens_CSV_File,
                    header=None,
                    names=TC_cols,
                    sep='\t',
                    decimal=','
                    )


    df.drop(df.tail(1).index,inplace=True) #drop last rows that can be not well written
    df.insert(0, 'Date and Time', pd.to_datetime(df['DD.MM.YYYY'] + ' ' + df['HH.MM.SS'], format=date_time_format, errors='coerce')) 
    df.drop(['DD.MM.YYYY', 'HH.MM.SS'], axis=1, inplace=True) 
    df = df.loc[:, ~df.columns.str.contains('Channel')]
    return df

def save_TC_period_csv(tc_df, configName, start_time, end_time):
    ''' Save a list of csv files corresponding to one configuration 
        during its specified time interval between start_time and end_time.
        Each csv of the list correspond to a hobo sensor.
    '''
    

    mask = (tc_df['Date and Time'] >= start_time) & (tc_df['Date and Time'] <= end_time)
    filtered_df = tc_df[mask]
        
    filtered_df.to_csv(f'THERMOCOUPLES/{configName}_TC.csv', index=False, sep='\t')

In [48]:
# Dataframe
TC_8_1 = create_TC_DF('20231106', '8.1')
TC_9_1 = create_TC_DF('20231108', '9.1')

# CSV
save_TC_period_csv(TC_9_1, 'floorConfig0', '2023-11-06 22:45:00', '2023-11-06 23:00:00')
save_TC_period_csv(TC_9_1, 'floorConfig1', '2023-11-07 00:45:00', '2023-11-07 01:00:00')
save_TC_period_csv(TC_9_1, 'floorConfig2a', '2023-11-07 16:45:00', '2023-11-07 17:00:00')
save_TC_period_csv(TC_8_1, 'floorConfig2b', '2023-11-05 05:25:00', '2023-11-05 05:40:00')
save_TC_period_csv(TC_8_1, 'floorConfig3', '2023-11-05 12:25:00', '2023-11-05 12:40:00')
save_TC_period_csv(TC_9_1, 'floorConfig4', '2023-11-08 00:45:00', '2023-11-08 01:00:00')
save_TC_period_csv(TC_9_1, 'floorConfig6', '2023-11-08 05:45:00', '2023-11-08 06:00:00')
